## OpenCV Connecting to a USB Camera or a Laptop Camera

OpenCV can automatically connect to your laptop's built in camera or your USB camera if you've installed that specific USB camera drivers. If you are not able to connect, please check out the troubleshooting tips here:

* https://github.com/opencv/opencv/issues/8471

In [ ]:
# Uncomment to install
# !pip install onnx
# !pip install onnxruntime

In [28]:
import torch
import torch.nn as nn
import  onnx
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

Device:  cpu


In [18]:
def convert_2_onnx(path, model, batch_size=1):
  """
    Convert a PyTorch model to ONNX format.

    Args:
        path (str): Path to save the ONNX model.
        model (nn.Module): PyTorch model to be converted.
        batch_size (int, optional): Batch size for the input. Default is 1.
    """
  dummy_input = torch.randn(batch_size, 3, 48, 48, requires_grad=True).to(DEVICE)
  model.eval() # set the model to inference mode

  # Export the model
  torch.onnx.export(model,
        dummy_input,
        path,
        input_names = ['input'],
        output_names = ['output'],
        dynamic_axes = {'input':{0:'batch_size'}, 
                        'output':{0:'batch_size'}})

In [29]:
class SqueezeExcitationBlock(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(SqueezeExcitationBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(in_channels, in_channels // reduction_ratio)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(in_channels // reduction_ratio, in_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Global average pooling
        out = self.global_avg_pool(x).squeeze(-1).squeeze(-1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = out.unsqueeze(-1).unsqueeze(-1)
        return x * out

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.se_block = SqueezeExcitationBlock(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Apply SE block
        out = self.se_block(out)

        # Adjust dimensions of residual if needed
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class SEResNet(nn.Module):
    def __init__(self, block, layers, num_classes=7):
        super(SEResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual blocks
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # Global average pooling and fully-connected layer
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = [block(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels

        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x, return_feats=False):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        out = self.fc(x)

        if return_feats:
            return out, x
        else:
            return out

model = SEResNet(BasicBlock, [4, 5, 6, 2], num_classes=7).to(DEVICE)

In [20]:
path_to_checkpoints = 'checkpoint.pth' # path to your checkpoints

In [21]:
model.load_state_dict(torch.load(path_to_checkpoints, map_location=torch.device(DEVICE))['model_state_dict'])


<All keys matched successfully>

In [22]:
path = 'FER2013.onnx'
convert_2_onnx(path, model)

C:\Users\Gustave MB\AppData\Local\Temp\ipykernel_8416\2919049628.py:103: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if return_feats:


In [23]:
onnx.checker.check_model(onnx.load(path))